## BUAD 313 -- Session 7/8 (Modeling/Solving Systematically) -- Factory Planning LP

In [85]:
import numpy as np
from gurobipy import Model, GRB, quicksum

In Session 7, we introduced `Step 0` in optimization model formulation, which is to identify the structure of the input data. We're going to start by defining index sets, and then loading in some data by reading it in from a file and manipulating it.

### Scalar Data in the Case

In [86]:
#scalar data from the case, just written by hand
max_storage = 100
end_stock = 50
hours_per_mach = 384
storage_cost = 0.75


### Defining Index Sets

In [87]:
#define a list of products labeled A through G
products = ['A', 'B', 'C', 'D', 'E', 'F', 'G']

#define a list of integers from 1 to 12, representing the months of the year
months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

#define a list of machines types: grinding, vertical drilling, horizontal, boring, and planing, abbreviated by first letter (capitalized)
machines = ['G', 'V', 'H', 'B', 'P']

#define a list of tasks: produce, sell, and store 
tasks = ['Produce', 'Sell', 'Store']

### Loading Indexed Data

In [88]:
# short indexed data 
# I just typed these out by hand for convenience.  
# Be careful with ordering!  Needs to match index sets above!
resources = [4, 2, 3, 1, 1]
unit_profits = [15, 9, 12, 6, 16.5, 13.5, 4.5] 

In [98]:
## In this cell we're going to read in some csv data for the process_requirements, demand, and machine_down
process_requirements = np.genfromtxt('process_requirements.csv', delimiter=',', skip_header=True)[:, 1:]
demand = np.genfromtxt('demand.csv', delimiter=',', skip_header=True)[:, 1:]
machine_down = np.genfromtxt('machine_down.csv', delimiter=',', skip_header=True)[:, 1:]


(5, 7)

### Converting to Dictionaries

The next step isn't strictly necessary and is a little wasteful in terms of inefficiency.  We could just use the numpy arrays directly.  But it's a good example of how to convert the data to dictionaries if you want to use them in the model.  And accessing data via dictionaries is sometimes more intuitive than accessing via array indexing.

In [107]:
#create the dictionaries here

seasons_dict = {'Winter': 'W', 'Spring': 'Sp', 'Summer': 'Su', 'Fall': 'F'}
demand_dict = {(months[i], products[j]): demand[i, j] for i in range(12) for j in range(7)}
# define machine_dict as a dictionary with keys as months and machines and values from machine_down
machine_dict = {(months[i], machines[j]): machine_down[i, j] for i in range(12) for j in range(5)}
# define the unit_profit_dict as  dictionary with keys as machines and values as from resources
unit_profit_dict = {products[i]: unit_profits[i] for i in range(7)}
# define resource_dict as a dictionary iwth keys as machines and values as from reosurces
resource_dict = {machines[i]: resources[i] for i in range(5)}
# define process_requirements_dict as a dictionary with keys as products and values as from process_requirements
proc_req_dict = {
    (products[i], machines[j]): process_requirements[j, i]  # Ensure correct indexing
    for i in range(len(products))  # Loop over products
    for j in range(len(machines))  # Loop over machines
}


In [93]:
#let's also add one more dictionary that will be useful later, which is the available machine hours for each machine and month, accounting for machine down time

# create a dictionary with keys as months and machines and 
# values as difference between the number of resources available minus the number of machines downf for that machine
available_hours_dict = {(months[m], machines[ma]): hours_per_mach * (resources[ma] - machine_down[m, ma]) for m in range(12) for ma in range(5)}

It's a good idea to print out your dictionaries (or parts of them) to make sure they look the way you intended.

In [64]:
# print out the available dict as atab separated table with months as the rows and machines as the columns
print('Available Hours')
print('\t' + '\t'.join(machines))
for month in months:
    print(month, end='\t')
    for machine in machines:
        print(available_hours_dict[month, machine], end='\t')
    print()




Available Hours
	G	V	H	B	P
1	1152.0	768.0	1152.0	384.0	384.0	
2	1536.0	768.0	384.0	384.0	384.0	
3	1536.0	768.0	1152.0	0.0	384.0	
4	1536.0	384.0	1152.0	384.0	384.0	
5	1152.0	384.0	1152.0	384.0	384.0	
6	1152.0	768.0	768.0	384.0	0.0	
7	1536.0	768.0	384.0	0.0	384.0	
8	1536.0	384.0	1152.0	384.0	384.0	
9	1152.0	768.0	1152.0	384.0	0.0	
10	1536.0	768.0	1152.0	0.0	384.0	
11	1536.0	384.0	1152.0	384.0	384.0	
12	1536.0	768.0	768.0	384.0	0.0	


Looks good! On to getting the full LP into Gurobi. These are the `Step 1`, `Step 2`, and `Step 3` in optimization model formulation that we know: decision variables, objective function, and constraints. Still though, we can be more systematic in our approach to this than we have been so far in class. For didactic purposes, let us actually add the constraints before the objective function. (*As long as they are all added, the order in which they are written does not matter.*)

In [103]:
#create a gurobi model called factory
factory = Model('factory')

# VARIABLES
x = factory.addVars(products, tasks, months, name='x', lb=0) # addVars (different from addVar) default is continuous

# CONSTRAINTS
for m in months:
    for p in products:
        factory.addConstr(x[p, 'Sell', m] <= demand_dict[m, p], name='Demand' + str(m) + str(p))

for m in months:
    for p in products:
        factory.addConstr(x[p, 'Store', m] <= max_storage, name='MaxStorage' + str(m) + str(p))

for p in products:
    factory.addConstr(x[p, 'Store', 12] == end_stock, name='EndStock' + str(p))

for m in months:
    for mach in machines:
        factory.addConstr(
            quicksum(proc_req_dict[p, mach] * x[p, 'Produce', m] for p in products) <= available_hours_dict[m, mach],
            name='MachineHours' + str(m) + str(mach)
        )

for m in months:
    for p in products:
        if m == 1:
            factory.addConstr(x[p, 'Store', m] == x[p, 'Produce', m] - x[p, 'Sell', m], name='FirstMonthStoreBalance' + str(p))
        else:
            factory.addConstr(x[p, 'Store', m-1] + x[p, 'Produce', m] - x[p, 'Sell', m] == x[p, 'Store', m], name='StoreBalance' + str(m) + str(p))

for m in months[1:]:
    for p in products:
        factory.addConstr(x[p, 'Store', m-1] + x[p, 'Produce', m] - x[p, 'Sell', m] == x[p, 'Store', m], name='StoreBalance' + str(m) + str(p))



# OBJECTIVE FUNCTION

# set the objective function to be the difference between
# the sum over all products j and months m of the unit profit of product p times the amount sold of product j in month m
# minus storage_cost times the sum over all products j and months m of the amount stored of product p in month m
factory.setObjective(
    quicksum(unit_profit_dict[p] * x[p, 'Sell', m] for p in products for m in months) - 
    storage_cost * quicksum(x[p, 'Store', m] for p in products for m in months), GRB.MAXIMIZE
)

# SOLVE

# optimize the model
factory.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[x86] - Darwin 23.5.0 23F79)



CPU model: Intel(R) Core(TM) i5-1038NG7 CPU @ 2.00GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 396 rows, 252 columns and 1052 nonzeros
Model fingerprint: 0x72938184
Coefficient statistics:
  Matrix range     [1e-02, 1e+00]
  Objective range  [8e-01, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+01, 2e+03]
Presolve removed 380 rows and 208 columns
Presolve time: 0.04s
Presolved: 16 rows, 44 columns, 58 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.0581250e+05   3.905000e+02   0.000000e+00      0s
       4    2.5908652e+05   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.06 seconds (0.00 work units)
Optimal objective  2.590865179e+05


In [104]:
# PRINT RESULTS

# print a tab separated tabled of the optimal values of the decision variables
print('Optimal Solution')
print('\t' + '\t'.join(products))
for m in months:
    print(m, end='\t')
    for p in products:
        print(x[p, 'Produce', m].x, end='\t')
    print()

    

Optimal Solution
	A	B	C	D	E	F	G
1	500.0	888.5714285714287	382.5	300.0	800.0	200.0	0.0	
2	700.0	600.0	117.5	0.0	500.0	300.0	250.0	
3	0.0	0.0	0.0	0.0	0.0	400.0	0.0	
4	200.0	300.0	400.0	500.0	200.0	0.0	100.0	
5	0.0	100.0	600.0	100.0	1100.0	300.0	100.0	
6	600.0	600.0	0.0	400.0	0.0	500.0	0.0	
7	0.0	0.0	100.0	0.0	0.0	500.0	0.0	
8	0.0	400.0	100.0	200.0	900.0	400.0	220.0	
9	400.0	100.0	0.0	0.0	0.0	0.0	0.0	
10	0.0	0.0	400.0	0.0	0.0	351.66666666666663	0.0	
11	600.0	200.0	400.0	250.0	1050.0	348.33333333333337	200.0	
12	450.0	1050.0	0.0	550.0	0.0	550.0	0.0	
